In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from bs4 import BeautifulSoup
import requests
import pandas as pd
!pip install ar_corrector
from ar_corrector.corrector import Corrector
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score
import nltk
nltk.download('all')
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords 

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [ ]:
BookName=[]
authers=[]
deps=[]
page_no=[]
publisher=[]
lang=[]
Links=[]
description=[]

In [ ]:
def filtering(word_list):
    stopword_list=stopwords.words('arabic')
    wordsfilter=[]
    st = ISRIStemmer()
    for a in word_list.split():
        if a in stopword_list:
            continue 
        stem = st.stem(a)
        wordsfilter.append(stem)
    return " ".join(wordsfilter)

In [ ]:
def deleteDublicatedNames(string):
        splitedName=[]
        name=string
        if 'كتب' in string.split():
            splitedName=string.split('كتب')
        elif 'الكاتب' in string.split():
            splitedName=string.split('الكاتب')
            #print(splitedName)
        splitedName = [i for i in splitedName if i]
            #print(splitedName)
        if len(splitedName)>0:
            name=splitedName[0]
        return " ".join(name.replace('كتب','').replace('الكاتب','').split())

In [ ]:
def AppendDataRecords(name,text):
    authers.append(deleteDublicatedNames(" ".join((text[0].text[12:]).replace('\n'," ").split())))
    lang.append(" ".join((text[2].text[11:]).split()))
    NoOfPages=(text[3].text[14:])
    numeric_filter=filter(str.isdigit,NoOfPages)
    numeric_string="".join(numeric_filter)
    if(numeric_string==''):
        numeric_string='10'
    page_no.append(int(numeric_string))
    publisher.append(" ".join((text[4].text[11:]).replace('دار','').replace('مؤسسة','').replace('\n'," ").split())) 
    BookName.append(" ".join(((name.replace('كتاب','')).replace('PDF','')).replace('\n'," ").split()))

In [ ]:
def no_OfRecords(ReadLink,name,text,desc):
    dep=((text[1].text[13:]).replace( 'تحميل','').replace('كتب','').replace('\n\n','\n'))
    ListOfDep=dep.split('\n')
    for x in ListOfDep:
        if x!='' :
            deps.append(" ".join(x.split()))
            Links.append(ReadLink)
            description.append(desc.replace('PDF',''))
            AppendDataRecords(name,text)

In [ ]:
url='https://www.arab-books.com'
corr = Corrector()
for i in range(1,40):
    #print('page'+str(i))
    req=requests.get(url+'/page/'+str(i)+'/')
    content=req.text
    soup=BeautifulSoup(content)
    ul=str(soup.findAll('ul', attrs={'class':'posts-items'}))
    ul=BeautifulSoup(ul)
    for a in ul.find_all('a',href=True, attrs={'class':'post-thumb'}):
        name=a.get('aria-label')
        link=a.get('href')
        req=requests.get(link)
        content=req.text
        content=BeautifulSoup(content)
        data=str(content.find_all('div', attrs={'class':'book-info'}))
        data=BeautifulSoup(data)
        parent=data.find('ul')
        text=parent.find_all("li")
        div=str(content.find_all('div',attrs={'class':'read-link-bottom'}))
        div=BeautifulSoup(div)
        link=div.find('a',href=True)
        if (not link):
            ReadLink='Not Avialable'
        else : ReadLink=link.get('href')
        div=str(content.find_all('div',attrs={'class':'entry-content entry clearfix'}))
        div=BeautifulSoup(div)
        rdiv=div.find_all('p')
        desc=" "
        if len(div.find_all('p'))>=1 :
            for doc in div.find_all('p'):
                if len(doc.text.split())>10:
                    desc=corr.contextual_correct(filtering((doc.text.replace('PDF',''))))
                    break
        no_OfRecords(ReadLink,name,text,desc)

In [ ]:
dic={'اسم الكتاب': BookName ,'الكاتب': authers , 'القسم': deps , 'عدد صفحات الكتاب': page_no , 'اللغه' : lang , 'دار النشر':publisher ,'رابط قراءه الكتاب' : Links ,'ملخص الكتاب' : description }
df = pd.DataFrame(dic)
df.to_csv('DataSet5.csv',index=False,header=True,encoding='utf-8-sig')

In [ ]:
df=pd.read_csv("DataSet5.csv",encoding= 'utf-8-sig')
df

,اسم الكتاب,الكاتب,القسم,عدد صفحات الكتاب,اللغه,دار النشر,رابط قراءه الكتاب,ملخص الكتاب
0,لغز القصر الاخضر,محمود سالم,روايات بوليسية,44,عربي,هنداوي,https://www.arab-books.com/books/%d9%83%d8%aa%...,نهض طاخ بكر نكر مرة خرى لبس تجر بسبب ما شق طرق...
1,لغز القفاز الاحمر,محمود سالم,روايات بوليسية,50,عربي,هنداوي,https://www.arab-books.com/books/%d9%83%d8%aa%...,يدم ختم طول فكر ؛ وضع قفز صغر جيب في هرع سجل د...
2,لغز القبر الملكي,محمود سالم,روايات بوليسية,52,عربي,هنداوي,https://www.arab-books.com/books/%d9%83%d8%aa%...,هذا كشف اثر وشك كشف ثري همي في فتح صفح تكن عرف...
3,لغز الكوخ المحترق,محمود سالم,روايات بوليسية,56,عربي,هنداوي,https://www.arab-books.com/books/%d9%83%d8%aa%...,يبدو غمر ثير ليل في كانت سعة اسم نصف ليل في وك...
4,لغز اللص الشبح,محمود سالم,روايات بوليسية,48,عربي,هنداوي,https://www.arab-books.com/books/%d9%83%d8%aa%...,مضى حتى بقة ساء فكر لغز في وكل حول يجد خرج وجد...
...,...,...,...,...,...,...,...,...
676,مجلة ميكي العدد 540,والت ديزني,الأطفال,17,عربي,نهضة مصر,https://www.arab-books.com/books/%d9%85%d8%ac%...,عبر كل يكي عدد 540 كتب لتر ديز كتب نول طفل شتي...
677,مجلة ميكي العدد 539,والت ديزني,الأطفال,30,عربي,نهضة مصر,https://www.arab-books.com/books/%d9%85%d8%ac%...,عبر كل يكي عدد 539 كتب لتر ديز كتب نول طفل شتي...
678,مجلة ميكي العدد 536,والت ديزني,الأطفال,30,عربي,نهضة مصر,https://www.arab-books.com/books/%d9%85%d8%ac%...,عبر كل يكي عدد 536 كتب لتر ديز كتب نول طفل شتي...
679,مجلة ميكي العدد 530,والت ديزني,الأطفال,26,عربي,نهضة مصر,https://www.arab-books.com/books/%d9%85%d8%ac%...,عبر كل يكي عدد 530 كتب لتر ديز كتب نول طفل شتي...


In [ ]:
columns=['اسم الكتاب','الكاتب','القسم','ملخص الكتاب']
df=df[columns]

In [ ]:
df['رقم القسم'] = df['القسم'].factorize()[0]
df['ملخص الكتاب'].replace(" ",np.nan,inplace=True)
df.dropna(subset=['ملخص الكتاب'],inplace=True)
prediction=[]
Accuracy_Score=[]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,اسم الكتاب,الكاتب,القسم,ملخص الكتاب,رقم القسم
0,لغز القصر الاخضر,محمود سالم,روايات بوليسية,نهض طاخ بكر نكر مرة خرى لبس تجر بسبب ما شق طرق...,0
1,لغز القفاز الاحمر,محمود سالم,روايات بوليسية,يدم ختم طول فكر ؛ وضع قفز صغر جيب في هرع سجل د...,0
2,لغز القبر الملكي,محمود سالم,روايات بوليسية,هذا كشف اثر وشك كشف ثري همي في فتح صفح تكن عرف...,0
3,لغز الكوخ المحترق,محمود سالم,روايات بوليسية,يبدو غمر ثير ليل في كانت سعة اسم نصف ليل في وك...,0
4,لغز اللص الشبح,محمود سالم,روايات بوليسية,مضى حتى بقة ساء فكر لغز في وكل حول يجد خرج وجد...,0
...,...,...,...,...,...
676,مجلة ميكي العدد 540,والت ديزني,الأطفال,عبر كل يكي عدد 540 كتب لتر ديز كتب نول طفل شتي...,17
677,مجلة ميكي العدد 539,والت ديزني,الأطفال,عبر كل يكي عدد 539 كتب لتر ديز كتب نول طفل شتي...,17
678,مجلة ميكي العدد 536,والت ديزني,الأطفال,عبر كل يكي عدد 536 كتب لتر ديز كتب نول طفل شتي...,17
679,مجلة ميكي العدد 530,والت ديزني,الأطفال,عبر كل يكي عدد 530 كتب لتر ديز كتب نول طفل شتي...,17


In [ ]:
count_vect=CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(df['اسم الكتاب'], df['رقم القسم'], test_size = 0.2, random_state = 0)
X_train_counts=count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.toarray()
nb= MultinomialNB()
nb.fit(X_train_tfidf, y_train)
nb.score(X_train_tfidf, y_train)
X_test_tfidf=count_vect.transform(X_test)
prediction.append(nb.predict(X_test_tfidf))
cm = confusion_matrix(y_test, prediction[0])
Accuracy_Score.append(accuracy_score(y_test, prediction[0]))
Accuracy_Score[0]

0.5564516129032258

In [ ]:
count_vect=CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(df['ملخص الكتاب'], df['رقم القسم'], test_size = 0.2, random_state = 0)
X_train_counts=count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.toarray()
nb= MultinomialNB()
nb.fit(X_train_tfidf, y_train)
nb.score(X_train_tfidf, y_train)
X_test_tfidf=count_vect.transform(X_test)
prediction.append(nb.predict(X_test_tfidf))
cm = confusion_matrix(y_test, prediction[1])
Accuracy_Score.append(accuracy_score(y_test, prediction[1]))
Accuracy_Score[1]

0.6370967741935484

In [ ]:
le = preprocessing.LabelEncoder()
x=[]
y=[]
x.append(le.fit_transform(df.iloc[:,0]))
x=np.transpose(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( x, df['رقم القسم'], test_size=0.2,random_state = 0)
gnb = GaussianNB()
gnb.fit(x_train, y_train)
prediction.append(gnb.predict(x_test))
Accuracy_Score.append(accuracy_score(y_test, prediction[2]))
Accuracy_Score[2]

0.5645161290322581

In [ ]:
Final_pred=[]
Max=-1
indx=0
for i in range(len(Accuracy_Score)):
    if Accuracy_Score[i]>Max:
        Max=Accuracy_Score[i]
        index=i
for i in range(len(prediction[0])):
    if prediction[0][i]==prediction[1][i]:
        Final_pred.append(prediction[0][i])
    elif prediction[0][i]==prediction[2][i]:
        Final_pred.append(prediction[0][i])
    elif prediction[1][i]==prediction[2][i]:
         Final_pred.append(prediction[0][i])
    else : Final_pred.append(prediction[index][i])
print(accuracy_score(y_test, Final_pred))

0.5564516129032258
